In [557]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score,mean_squared_error, SCORERS
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC,SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import roc_auc_score

In [558]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [559]:
df = pd.read_excel('/content/drive/MyDrive/datasets/test_zad/Dataset_scoring.xlsx')
df

,ID,SEX,CHILD_BOOL,REALTY_BOOL,HAVE_ACCOUNT_BOOL,EMAIL_BOOL,DCI,PROFIT_FAMILY,UCI,FOREIGN_PASSPORT_BOOL,SIML,EDUCATION,TRANSPORT_AMOUNT,target,sample
0,2,1,1,1,0,0,28312.8594,33312.8594,0.0,1,19,3,2,1.0,Train
1,4,0,0,0,0,0,28738.4707,28738.4707,0.0,0,20,3,0,1.0,Train
2,6,1,1,1,0,0,25920.5195,37000.0000,0.0,0,38,3,0,1.0,Train
3,7,1,0,1,1,0,0.0000,16243.8096,7000.0,0,71,5,1,1.0,Train
4,8,1,1,1,0,0,17400.0000,17400.0000,0.0,0,59,3,3,1.0,Train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2157,2153,1,0,0,0,0,14048.3604,14048.7998,0.0,0,10,4,1,NaN,Test
2158,2154,1,1,1,0,0,6983.8799,23000.0000,15000.0,1,46,5,0,NaN,Test
2159,2155,0,1,1,1,0,23226.2402,45000.0000,0.0,0,36,3,0,NaN,Test
2160,2158,1,0,0,0,0,12006.0000,12000.0000,0.0,0,8,3,0,NaN,Test


# Train

In [560]:
#from jupyter notebook
# df = pd.read_excel('Dataset_scoring.xlsx')
# df

In [561]:
del df['sample']
test_x=df[df['target'].isnull()]
ID=test_x['ID']
del test_x['ID']
del test_x['target']

In [562]:
cor=df.corr()
cor=cor['target']
cor

ID                      -0.865959
SEX                      0.086680
CHILD_BOOL              -0.160028
REALTY_BOOL             -0.196304
HAVE_ACCOUNT_BOOL        0.082435
EMAIL_BOOL              -0.102050
DCI                     -0.126488
PROFIT_FAMILY           -0.038960
UCI                     -0.074196
FOREIGN_PASSPORT_BOOL   -0.213821
SIML                    -0.193122
EDUCATION                0.224297
TRANSPORT_AMOUNT        -0.177569
target                   1.000000
Name: target, dtype: float64

In [563]:
w=[]
for i in df['UCI']:
    if i==0:
        w.append(df['UCI'].mean())
    else:
        w.append(i)
        

In [564]:
h=[]
for i in df['DCI']:
     if i==0:
         h.append(df['DCI'].mean())
     else:
         h.append(i)

In [565]:
W=pd.Series(w)
H=pd.Series(h)

In [566]:
#rework UCI
df = pd.concat([df, W], axis=1)

In [567]:
del df['UCI']

In [568]:
df.rename(columns={ df.columns[-1]: "UCI", }, inplace = True)

In [569]:
#rework DCI
df = pd.concat([df, H], axis=1)

df.rename(columns={ df.columns[-1]: "DCI", }, inplace = True)

In [570]:
del df['DCI']

In [571]:
train=df[df['target'].isin([0,1])]
train=train.sample(frac=1) 
del train['ID']

In [572]:
train

,SEX,CHILD_BOOL,REALTY_BOOL,HAVE_ACCOUNT_BOOL,EMAIL_BOOL,PROFIT_FAMILY,FOREIGN_PASSPORT_BOOL,SIML,EDUCATION,TRANSPORT_AMOUNT,target,UCI
669,1,1,0,0,0,29708.7305,0,8,5,0,1.0,4280.579867
1191,0,1,0,0,0,20000.0000,0,19,5,0,0.0,4280.579867
106,0,0,1,0,0,10109.0000,0,9,6,0,1.0,5000.000000
723,0,1,0,0,0,24473.4297,0,21,5,0,1.0,4280.579867
1104,0,1,0,0,0,17184.7207,0,46,5,0,0.0,4280.579867
...,...,...,...,...,...,...,...,...,...,...,...,...
407,1,0,0,0,0,9666.0703,0,12,4,0,1.0,4280.579867
785,0,1,0,0,0,10000.0000,1,74,5,0,0.0,4280.579867
917,0,0,1,0,0,22587.9902,0,84,3,0,0.0,10000.000000
361,1,1,0,1,0,35000.0000,0,0,3,1,1.0,4280.579867


In [573]:
def OneHotEncoder(df):
    '''Функция производит Encoding'''
    count = 0
    for i in df.columns:
        if i == 'EDUCATION':
            df = pd.concat([df, pd.get_dummies(df[i], prefix=(i+'M'))], axis=1)
            df.drop(i, axis=1, inplace=True)
    return df

In [574]:
train=OneHotEncoder(train)
train

,SEX,CHILD_BOOL,REALTY_BOOL,HAVE_ACCOUNT_BOOL,EMAIL_BOOL,PROFIT_FAMILY,FOREIGN_PASSPORT_BOOL,SIML,TRANSPORT_AMOUNT,target,UCI,EDUCATIONM_1,EDUCATIONM_2,EDUCATIONM_3,EDUCATIONM_4,EDUCATIONM_5,EDUCATIONM_6,EDUCATIONM_7
669,1,1,0,0,0,29708.7305,0,8,0,1.0,4280.579867,0,0,0,0,1,0,0
1191,0,1,0,0,0,20000.0000,0,19,0,0.0,4280.579867,0,0,0,0,1,0,0
106,0,0,1,0,0,10109.0000,0,9,0,1.0,5000.000000,0,0,0,0,0,1,0
723,0,1,0,0,0,24473.4297,0,21,0,1.0,4280.579867,0,0,0,0,1,0,0
1104,0,1,0,0,0,17184.7207,0,46,0,0.0,4280.579867,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,1,0,0,0,0,9666.0703,0,12,0,1.0,4280.579867,0,0,0,1,0,0,0
785,0,1,0,0,0,10000.0000,1,74,0,0.0,4280.579867,0,0,0,0,1,0,0
917,0,0,1,0,0,22587.9902,0,84,0,0.0,10000.000000,0,0,1,0,0,0,0
361,1,1,0,1,0,35000.0000,0,0,1,1.0,4280.579867,0,0,1,0,0,0,0


In [575]:
del train['PROFIT_FAMILY']

In [576]:
cor=train.corr()
cor=cor['target']

cor
#DCI 11.67 with <600000
#DCI 11.36 without <600000
#DCI 12.5 with <300000

SEX                      0.086680
CHILD_BOOL              -0.160028
REALTY_BOOL             -0.196304
HAVE_ACCOUNT_BOOL        0.082435
EMAIL_BOOL              -0.102050
FOREIGN_PASSPORT_BOOL   -0.213821
SIML                    -0.193122
TRANSPORT_AMOUNT        -0.177569
target                   1.000000
UCI                     -0.085033
EDUCATIONM_1            -0.029264
EDUCATIONM_2            -0.052983
EDUCATIONM_3            -0.233880
EDUCATIONM_4             0.154170
EDUCATIONM_5             0.010425
EDUCATIONM_6             0.161991
EDUCATIONM_7             0.048653
Name: target, dtype: float64

In [577]:
# del train['EDUCATIONM_7']
del train['EDUCATIONM_1']
del train['EDUCATIONM_5']


In [578]:
# del train['EDUCATIONM_7']

In [579]:
cor=train.corr()
cor=cor['target']
cor

SEX                      0.086680
CHILD_BOOL              -0.160028
REALTY_BOOL             -0.196304
HAVE_ACCOUNT_BOOL        0.082435
EMAIL_BOOL              -0.102050
FOREIGN_PASSPORT_BOOL   -0.213821
SIML                    -0.193122
TRANSPORT_AMOUNT        -0.177569
target                   1.000000
UCI                     -0.085033
EDUCATIONM_2            -0.052983
EDUCATIONM_3            -0.233880
EDUCATIONM_4             0.154170
EDUCATIONM_6             0.161991
EDUCATIONM_7             0.048653
Name: target, dtype: float64

In [580]:
train_y=train['target']

In [581]:
train_x=train
del train_x['target']

In [582]:
train_x

,SEX,CHILD_BOOL,REALTY_BOOL,HAVE_ACCOUNT_BOOL,EMAIL_BOOL,FOREIGN_PASSPORT_BOOL,SIML,TRANSPORT_AMOUNT,UCI,EDUCATIONM_2,EDUCATIONM_3,EDUCATIONM_4,EDUCATIONM_6,EDUCATIONM_7
669,1,1,0,0,0,0,8,0,4280.579867,0,0,0,0,0
1191,0,1,0,0,0,0,19,0,4280.579867,0,0,0,0,0
106,0,0,1,0,0,0,9,0,5000.000000,0,0,0,1,0
723,0,1,0,0,0,0,21,0,4280.579867,0,0,0,0,0
1104,0,1,0,0,0,0,46,0,4280.579867,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,1,0,0,0,0,0,12,0,4280.579867,0,0,1,0,0
785,0,1,0,0,0,1,74,0,4280.579867,0,0,0,0,0
917,0,0,1,0,0,0,84,0,10000.000000,0,1,0,0,0
361,1,1,0,1,0,0,0,1,4280.579867,0,1,0,0,0


In [583]:
s_scaler = StandardScaler()
scaled_X = s_scaler.fit_transform(train_x) 
x_train, x_test, y_train, y_test = train_test_split(scaled_X, train_y.to_numpy())

In [584]:
clf_0 = KNeighborsClassifier()
params = {
    'n_neighbors': [1,3,5,10,20,30,40,50,60],
    'weights': ['uniform', 'distance'],
    'p': [1,2,3,4,5]
}
c_0 = GridSearchCV(clf_0, 
                  param_grid=params, 
                  scoring='roc_auc', 
                  cv=10, # делим на 5 частей
                  refit=True, # Выберет самую хорошую модель и применит на тестовуб
                  n_jobs=4,# сколько потоков считать
                  verbose=5)
c_0.fit(scaled_X, train_y)
c_0.best_params_, c_0.best_score_
#61.55 Start, without feat
#71.80 standartscaler+
#71.58 del UCI,Profit_family
#72.21 OneHotEncoder +
#72.42 del EDU1,EDU5 +
#72.51 Rework UCI +
#72.62 rework DCI,UCI +
#72.7 del UCI rework DCI +
#72.54 rework UCI,DCI <600000 -
#del EDU7+

Fitting 10 folds for each of 90 candidates, totalling 900 fits


({'n_neighbors': 50, 'p': 1, 'weights': 'uniform'}, 0.7270641735918744)

In [585]:
clf_1 = LogisticRegression(max_iter=1000)
params = {'C':np.arange(0,1,.1)}
c_1 = GridSearchCV(clf_1,
                  param_grid=params,
                  scoring='roc_auc', 
                  cv=10, # делим на 5 частей
                  refit=True, # Выберет самую хорошую модель и применит на тестовуб
                  n_jobs=4,# сколько потоков считать
                  verbose=5)
c_1.fit(scaled_X, train_y)
c_1.best_params_, c_1.best_score_
#64.04 start without feat
#72.50 standartscaler +
#72.32 del UCI,Profit_family -
#73.59 OneHotEncoder +
#73.54 del EDU1,EDU5 +
#73.37 Rework UCI -
#73.61 Rework DCI,UCI +
#73.51 Rework DCI del UCI -

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1614, in fit
    for class_, warm_start_coef_ in zip(classes_, warm_start_coef)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/loc

({'C': 0.9}, 0.7362411126500461)

In [586]:
clf_2 = SVC()
params = {'C':np.arange(0,1,0.1)}
c_2= GridSearchCV(clf_2,
                  param_grid=params,
                  scoring='roc_auc', 
                  cv=10, # делим на 5 частей
                  refit=True, # Выберет самую хорошую модель и применит на тестовуб
                  n_jobs=4,# сколько потоков считать
                  verbose=5)
c_2.fit(scaled_X, train_y)
c_2.best_params_, c_2.best_score_
#62.81 start,without feat
#71.44 standartscaler=
#71.62 del UCI,Profit_family
#72.12 OneHotEncoder +
#72.21 del EDU1,EDU5 +
#72.299 Rework UCI +
#72.40 Rework DCI,UCI -
#72.36 Rework DCI del UCI +

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py", line 333, in _dense_fit
    random_seed=random_seed,
  File "sklearn/svm/_libsvm.pyx", line 

({'C': 0.1}, 0.726768351800554)

In [587]:
clf_4 = DecisionTreeClassifier()
params = {'max_depth':np.arange(1,10,1),
          'min_samples_split':np.arange(1,10,1)}
c_4 = GridSearchCV(clf_4,
                  param_grid=params,
                  scoring='roc_auc', 
                  cv=10, # делим на 5 частей
                  refit=True, # Выберет самую хорошую модель и применит на тестовуб
                  n_jobs=4,# сколько потоков считать
                  verbose=5)
c_4.fit(scaled_X, train_y)

c_4.best_params_, c_4.best_score_
#67.78 start,without feat
#69.28 standartscaler+
#69.59 del UCI,Profit_family
#67.94 OneHotEncoder -
#67.69 del EDU1, EDU5 -
#68.37 Rework UCI +
#69.43 Rework DCI,UCI +
#67.96 Rework DCI del UCI -  
#67.59 -

Fitting 10 folds for each of 81 candidates, totalling 810 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
90 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 254, in fit
    % self.min_samples_split
ValueError: min_samples_split must be an integer greater than 1 or a float 

({'max_depth': 6, 'min_samples_split': 3}, 0.6836320406278855)

In [588]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


RF_1 = RandomForestClassifier()
params = {'n_estimators': [100,200,300],
          'max_depth': [3,4,5,6,7,8,9],
          'min_samples_split':[4,5,6,7,8,9]}
RF = GridSearchCV(RF_1,
                  param_grid=params,
                  scoring='roc_auc' ,
                  cv=10, # делим на 5 частей
                  refit=True, # Выберет самую хорошую модель и применит на тестовуб
                  n_jobs=4,# сколько потоков считать
                  verbose=5)

In [589]:
model_1 = RF.fit(scaled_X,train_y)
model_1.best_params_, model_1.best_score_
#72.00 start, without feat
#71.63 standartscaler-
#71.72 del UCI,Profit_family +
#72.2 OneHotEncoder +
#72.3 del EDU1,EDU5 +
#72.226 Rework UCI -
#72.87 Rework DCI,UCI +
#72.29 rework DCI del UCI -
#72.53 Rework DCI,UCI <600000 +

Fitting 10 folds for each of 126 candidates, totalling 1260 fits


({'max_depth': 5, 'min_samples_split': 6, 'n_estimators': 200},
 0.7290216989843028)

In [450]:
!pip install xgboost

In [451]:
import xgboost
from xgboost import XGBClassifier

In [453]:
xgboost = XGBClassifier(learning_rate=0.0009,
                       n_estimators=4750,
                       max_depth=4
                       )

In [454]:
model_2 = xgboost.fit(x_train,y_train)

In [455]:
y_pred = model_2.predict(x_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
roc_auc = roc_auc_score(y_test, predictions)
print("Roc_auc_score: %.2f%%" % (roc_auc * 100.0))
#64.10

Roc_auc_score: 62.40%


In [456]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 14 kB/s 


In [457]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [458]:
lightboost = LGBMClassifier(learning_rate=0.0009,
                       n_estimators=4750,
                       max_depth=4
                       )

In [459]:
model_3 = lightboost.fit(x_train,y_train)
y_pred = model_3.predict(x_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = roc_auc_score(y_test, predictions)
print("Roc_auc_score: %.2f%%" % (accuracy * 100.0))
#63.26

Roc_auc_score: 64.18%


In [460]:
Cat = CatBoostClassifier(learning_rate=0.0009,
                       n_estimators=4750,
                       max_depth=4
                       )

In [461]:
model_4 = Cat.fit(x_train,y_train)
y_pred = model_4.predict(x_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = roc_auc_score(y_test, predictions)
print("Roc_auc_score: %.2f%%" % (accuracy * 100.0))
#69.13
#63.35

0:	learn: 0.6929939	total: 47.2ms	remaining: 3m 44s
1:	learn: 0.6928909	total: 48ms	remaining: 1m 53s
2:	learn: 0.6927659	total: 48.7ms	remaining: 1m 17s
3:	learn: 0.6926601	total: 49.7ms	remaining: 59s
4:	learn: 0.6925445	total: 50.7ms	remaining: 48.1s
5:	learn: 0.6923956	total: 51.6ms	remaining: 40.8s
6:	learn: 0.6922649	total: 52.8ms	remaining: 35.8s
7:	learn: 0.6921454	total: 53.8ms	remaining: 31.9s
8:	learn: 0.6920006	total: 54.7ms	remaining: 28.8s
9:	learn: 0.6918719	total: 55.7ms	remaining: 26.4s
10:	learn: 0.6917495	total: 56.6ms	remaining: 24.4s
11:	learn: 0.6916198	total: 57.5ms	remaining: 22.7s
12:	learn: 0.6915010	total: 58.6ms	remaining: 21.3s
13:	learn: 0.6913719	total: 59.6ms	remaining: 20.2s
14:	learn: 0.6912705	total: 60.6ms	remaining: 19.1s
15:	learn: 0.6911409	total: 61.5ms	remaining: 18.2s
16:	learn: 0.6909995	total: 62.4ms	remaining: 17.4s
17:	learn: 0.6908543	total: 63.3ms	remaining: 16.6s
18:	learn: 0.6907382	total: 64.2ms	remaining: 16s
19:	learn: 0.6906186	tota

In [462]:
prediction=model_4.predict(test_x)
prediction

CatBoostError: ignored

In [ ]:
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier

clf_0 = KNeighborsClassifier()
params = {
    'n_neighbors': [1,3,5,10,20,30,40,50,60],
    'weights': ['uniform', 'distance'],
    'p': [1,2,3,4,5]
}
knn1 = GridSearchCV(clf_0, 
                  param_grid=params, 
                  scoring='accuracy', 
                  cv=10, # делим на 5 частей
                  refit=True, # Выберет самую хорошую модель и применит на тестовуб
                  n_jobs=5,# сколько потоков считать
                  verbose=5)
knn1.fit(x_train, y_train)

clf_1 = LogisticRegression(max_iter=1000)
params = {'C':np.arange(0,1.5,.1)}
LR = GridSearchCV(clf_1,
                  param_grid=params,
                  scoring='accuracy', 
                  cv=10, # делим на 10 частей
                  refit=True, # Выберет самую хорошую модель и применит на тестовуб
                  n_jobs=5,# сколько потоков считать
                  verbose=5)
lr1 = LR.fit(x_train, y_train)


RF_1 = RandomForestClassifier()
params = {'n_estimators': [100,200,300],
          'max_depth': [4,5,6,7,8,9],
          'min_samples_split':[4,5,6,7,8]}
RF = GridSearchCV(RF_1,
                  param_grid=params, 
                  cv=8, # делим на 8 частей
                  refit=True, # Выберет самую хорошую модель и применит на тестовуб
                  n_jobs=4,# сколько потоков считать
                  verbose=5)
rf1 = RF.fit(train_x,train_y)

Fitting 10 folds for each of 90 candidates, totalling 900 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=5)]: Done 198 tasks      | elapsed:    4.8s
[Parallel(n_jobs=5)]: Done 558 tasks      | elapsed:   15.2s
[Parallel(n_jobs=5)]: Done 836 tasks      | elapsed:   25.1s
[Parallel(n_jobs=5)]: Done 900 out of 900 | elapsed:   28.5s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.0s


Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=5)]: Done 150 out of 150 | elapsed:    0.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 8 folds for each of 90 candidates, totalling 720 fits


[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    3.0s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   20.3s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   51.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:  3.8min
[Parallel(n_jobs=4)]: Done 720 out of 720 | elapsed:  4.4min finished


In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

class DjStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict(valid)
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)

In [ ]:
from xgboost import XGBClassifier
models = [knn1, lr1, rf1]
ens_model = XGBClassifier(learning_rate=0.0009,
                       n_estimators=4750,
                       max_depth=4
                       )
s1 = DjStacking(models, ens_model)
s1.fit(x_train, y_train)
y_pred = s1.predict(x_test)
predictions = [round(value) for value in y_pred]
roc_auc = roc_auc_score(y_test, predictions)
print("Roc_auc_score: %.2f%%" % (roc_auc * 100.0))


Fitting 10 folds for each of 90 candidates, totalling 900 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done 198 tasks      | elapsed:    2.2s
[Parallel(n_jobs=5)]: Done 558 tasks      | elapsed:    5.6s
[Parallel(n_jobs=5)]: Done 900 out of 900 | elapsed:    9.7s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.1s


Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=5)]: Done 150 out of 150 | elapsed:    2.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 8 folds for each of 90 candidates, totalling 720 fits


[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.7s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   18.7s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   46.9s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  2.3min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:  3.4min
[Parallel(n_jobs=4)]: Done 720 out of 720 | elapsed:  3.9min finished


Roc_auc_score: 66.19%
